# Lab3.4 Sentiment Classification using transformer models

Copyright: Vrije Universiteit Amsterdam, Faculty of Humanities, CLTL

This notebook explains how you can use a transformer model that is fine-tuned for sentiment analysis. Fine-tuned transformer models are published regularly on the huggingface platform: https://huggingface.co/models

These models are very big (Gigabytes) and require a computer with sufficient memory to load. Furthermore, loading these models takes some time as well. It is also possible to copy such a model to your disk and to load the local copy. Still a substantial memory is needed to load it.

This notebook requires installing some deep learning packages: transformers, pytorch and simpletransformers. If you are not experienced with installing these packages, make sure you first define a virtual environment for python, activate this environment and install the packages in this enviroment.

Please consult the Python documentation for installing such an enviroment:

https://docs.python.org/3/library/venv.html

After activating your enviroment you can install pytorch, transformers and simpletransformers from the command line. If you start this notebook within the same virtual environment you can also execute the next installation commands from your notebook. Once installed, you can comment out the next cell.

In [1]:
# !conda install pytorch cpuonly -c pytorch
# !pip install transformers
# !pip install simpletransformers

Huggingface transfomers provides an option to create a **pipeline** to perform a NLP task with a pretrained model: 

"The pipelines are a great and easy way to use models for inference. These pipelines are objects that abstract most of the complex code from the library, offering a simple API dedicated to several tasks, including Named Entity Recognition, Masked Language Modeling, Sentiment Analysis, Feature Extraction and Question Answering."

More information can be found here: https://huggingface.co/transformers/v3.0.2/main_classes/pipelines.html

We will use the pipeline module to load a fine-tuned model to perform senteiment analysis

In [2]:
from transformers import pipeline

c:\Users\amina\anaconda3\envs\text-mining\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


We load a transformer model 'distilbert-base-uncased-finetuned-sst-2-english' that is fine-tuned for binary classification from the Hugging face repository:

https://huggingface.co/models

We need to load the model for the sequence classifcation and the tokenizer to convert the sentences into tokens according to the vocabulary of the model.

Loading the model takes some time.

In [3]:
sentimentenglish = pipeline("sentiment-analysis", 
                            model="distilbert-base-uncased-finetuned-sst-2-english", 
                            tokenizer="distilbert-base-uncased-finetuned-sst-2-english")

c:\Users\amina\anaconda3\envs\text-mining\Lib\site-packages\huggingface_hub\file_download.py:142: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\amina\.cache\huggingface\hub\models--distilbert-base-uncased-finetuned-sst-2-english. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Device set to use cpu


We now created an instantiation of a pipeline that can tokenize any sentence, obtain a sententence embedding from the transformer language model and perform the **sentiment-analysis** task. Let's try it out on an example sentence.

In [4]:
sentence_pos_en = "Nice hotel and the service is great"

In [5]:
sentimentenglish(sentence_pos_en)

[{'label': 'POSITIVE', 'score': 0.999881386756897}]

In [6]:
sentence_neg_en = "The rooms are dirty and the wifi does not work"

In [7]:
sentimentenglish(sentence_neg_en)

[{'label': 'NEGATIVE', 'score': 0.9997870326042175}]

This is easy and seems to work very well. 

## Using a Dutch fine-tuned transformer model

We can use a fine-tuned Dutch model for Dutch sentiment analysis by creating another pipeline. Again loading this model takes some time. Also note that after loading, both moodels are loaded in memory. So if you have issues loading, you may want to start over and try again just with the Dutch pipeline.

In [8]:
sentimentdutch = pipeline("sentiment-analysis", 
                          model="wietsedv/bert-base-dutch-cased-finetuned-sentiment", 
                          tokenizer="wietsedv/bert-base-dutch-cased-finetuned-sentiment")

c:\Users\amina\anaconda3\envs\text-mining\Lib\site-packages\huggingface_hub\file_download.py:142: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\amina\.cache\huggingface\hub\models--wietsedv--bert-base-dutch-cased-finetuned-sentiment. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Device set to use cpu


We test it on two similar Dutch sentences:

In [9]:
sentence_pos_nl="Mooi hotel en de service is geweldig"
sentence_neg_nl="De kamers zijn smerig en de wifi doet het niet"

In [10]:
sentimentdutch(sentence_pos_nl)

[{'label': 'pos', 'score': 0.9999955892562866}]

In [11]:
sentimentdutch(sentence_neg_nl)

[{'label': 'neg', 'score': 0.6675444841384888}]

This seems to work fine too although the score for negative in the second example is much lower.

## Inspecting sentence representations using Simpletransformers

The Simpletransformers package is built on top of the transformer package. It simplifies the use of transformers even more and provides excellent documentation: https://simpletransformers.ai

The site explains also how you can fine-tune models yourself or even how to build models from scratch, assuming you have the computing power and the data.

Here we are going to use it to inspect the sentence representations a bit more. Unfortunately, we need to load the English model again as an instantiation of a RepresentationModel. So if you have memory issues, please stop the kernel and start again from here.

Loading the model may gave a lot of warnings. You can ignore these. If you do not have a graphical card (GPU) and or cuda installed to use the GPU you need to set use_cuda to False, as shown below.

In [12]:
from simpletransformers.language_representation import RepresentationModel
        
#sentences = ["Example sentence 1", "Example sentence 2"]
model = RepresentationModel(
        model_type="bert",
        model_name="distilbert-base-uncased-finetuned-sst-2-english",
        use_cuda=False ## If you cannot use a GPU set this to false
    )

You are using a model of type distilbert to instantiate a model of type bert. This is not supported for all configurations of models and can yield errors.
Some weights of BertForTextRepresentation were not initialized from the model checkpoint at distilbert-base-uncased-finetuned-sst-2-english and are newly initialized: ['embeddings.LayerNorm.bias', 'embeddings.LayerNorm.weight', 'embeddings.position_embeddings.weight', 'embeddings.token_type_embeddings.weight', 'embeddings.word_embeddings.weight', 'encoder.layer.0.attention.output.LayerNorm.bias', 'encoder.layer.0.attention.output.LayerNorm.weight', 'encoder.layer.0.attention.output.dense.bias', 'encoder.layer.0.attention.output.dense.weight', 'encoder.layer.0.attention.self.key.bias', 'encoder.layer.0.attention.self.key.weight', 'encoder.layer.0.attention.self.query.bias', 'encoder.layer.0.attention.self.query.weight', 'encoder.layer.0.attention.self.value.bias', 'encoder.layer.0.attention.self.value.weight', 'encoder.layer.0.interme

The Representationmodel allows you to obtain a sentence encoding. We do that next for the positive English example which consists of 7 words:

In [13]:
sentence_pos_en

'Nice hotel and the service is great'

According to the simpletransformers API the input must be a list even when it is a single sentence. If you pass a string as input, it will turn it into a list of charcaters, each character as a separate sentence.

In [14]:
word_vectors = model.encode_sentences([sentence_pos_en], combine_strategy=None)

The result is a numpy array with the shape (1, 9, 768) 

In [15]:
print(type(word_vectors))
print(word_vectors.shape)

<class 'numpy.ndarray'>
(1, 9, 768)


The first number indicates the number of sentences, which is **1** in our case. The next digit **9** indicates the number of tokens and the final digit is the number of dimension for each token according to the transformer model, which **768** in case of BERT models.

We can ask for the full embedding representation for the first token:

In [16]:
print('Nr of dimensions for the mebdding of the first token:', len(word_vectors[0][0]))
print(word_vectors[0][0])

Nr of dimensions for the mebdding of the first token: 768
[-1.5398697  -1.0169228   1.0272177   1.2418951   1.2692713   0.87777656
 -0.7223468   1.3373877  -2.8084295   0.1980253  -0.8346974  -0.9953711
 -2.032871   -0.3409271   2.139137    1.9258752   0.85766643  2.5137255
  0.67765296 -0.41850653 -1.4893218  -0.17095388  1.1302085   1.3729486
  1.1725357  -0.21008687 -0.9759953  -1.3556827  -1.7344168  -0.6167605
 -1.2852031   0.61838925 -0.1619733   0.59804755 -0.4538428   0.6089023
 -1.1695803   0.08239149 -0.807556   -1.0054197  -0.48749292 -0.3784391
  1.0347805   0.14620247  0.18716033 -1.4329507   0.49156392 -0.32463717
  2.1962826   0.4746712   0.03687827  0.31791258  1.8357697   0.6402055
  1.5319984   1.3642994   0.29719186  1.2101345   0.41975528 -0.82035357
 -0.07322589 -0.89692205  0.19344972  1.1281337   0.8287384  -0.7509966
 -0.58949244  0.27243912 -0.11436398  0.7038807  -0.6747096   0.8578785
  0.79638165 -0.16919458  0.89852536 -1.1970075   1.8811995   0.64701116
 -

**WAIT** Our sentence has 7 words so why do we get 9 tokens here?

We can  use the tokenizer of the model to get the token representation of the transformer and check it out.

In [17]:
tokenized_sentence = model.tokenizer(sentence_pos_en)
tokenized_sentence

{'input_ids': [101, 100, 3309, 1998, 1996, 2326, 2003, 2307, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1]}

Although our sentence has 7 words, we get 9 identifiers. We can use the **decode** function to convert them back to words:

In [18]:
model.tokenizer.decode(101)

'[CLS]'

The first token is the special token **CLS** which is an abstract sentence representation. Let's check another one:

In [19]:
model.tokenizer.decode(3309)

'hotel'

Allright, this a word from our sentence. Let's decode them all:

In [20]:
tokenid_list = tokenized_sentence['input_ids']
for token_id in tokenid_list:
    print(token_id, model.tokenizer.decode(token_id))

101 [CLS]
100 [UNK]
3309 hotel
1998 and
1996 the
2326 service
2003 is
2307 great
102 [SEP]


The transformer model added the special tokens **CLS** and **SEP** but also represented our "Nice" with the **UNK** token. Any idea why? Check the name of the model we used.....

We used the uncased model, which means that for training all inoput was downcased.

# End of this notebook